In [5]:
import ee
import folium

from eegeolab.bmath import Ratio

In [6]:
ee.Initialize()

In [7]:
ratio = Ratio('VV', 'VH')
ratio.__dict__

{'num': 'VV', 'denom': 'VH', '_name': 'VV/VH'}

In [8]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [12]:
lat, lon = 37.9018, -122.292
point = ee.Geometry.Point(lon, lat)

s1 = ee.ImageCollection('COPERNICUS/S1_GRD')

image = ee.Image(
  s1.filterBounds(point)
    .filterDate('2015-01-01', '2015-12-31')
    .first()
)

# Calculate Ratio Cap
compute = ratio.compute(image=image)

In [15]:
my_map = folium.Map(location=[lat, lon], zoom_start=9)
vis = {'min': -1, 'max': 1, 'bands': ['VV/VH']}

# Add the land cover to the map object.
my_map.add_ee_layer(compute, vis, 'components')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)